# Investgating Data Collection Progress Thus Far

i.e. 2023-03-30.

Refer to [2023-03-30_logbook](../../001_obsidian_vault/2023-03-31_logbook.md).

## 1. What Runs are Appropriate for Analysis.

Criteria: 

1. were on the Avantor 10cm column.
2. contain UV spectra.
3. Not uracil, acetone or coffee runs.

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
from pathlib import Path

from agilette.modules.library import Library


In [ ]:
lib = Library('/Users/jonathan/0_jono_data')

Let's try selecting only for rows that contain UV data.


In [ ]:
meta_df = lib.metadata_table
meta_df[['ch_filenames', 'uv_filenames']]

In [ ]:
avantor_df = meta_df[(meta_df['uv_filenames']!='') & (meta_df['acq_method'].str.contains('AVANTOR')) & ~(meta_df['name'].str.contains('uracil')) & ~(meta_df['name'].str.contains('coffee')) & ~(meta_df['name'].str.contains('lor'))]

avantor_df.shape

In [ ]:
avantor_df

In [ ]:
avantor_df.describe()

70 unique samples doesn't seem too bad. There should be more to pull sitting in the instrument as well. It appears that this filter is legitimate, as there is UV data for every remaining row.

Now, one sticking point is that we don't have the wine names in this table. To get them we will need to load the sample tracker as a df and merge them on the sample ID., or 'name' in this dataframe. A further complicating factor is that the names are sometimes not consistant with sample tracker, for example '2021-debortoli-cabernet-merlot_avantor`. I dont believe it was ever added, and it makes up 7/107 of the runs. Let's first load sample_tracker then compare their contents.